In [1]:
from prefect import Flow,task, Task,unmapped, Parameter
import sys
import os
from os.path import join, splitext
import matplotlib.pyplot as plt
from pandas import to_datetime
import xarray as xr
from scipy import ndimage, integrate

In [2]:

import plotly.graph_objects as go


import plotly
plotly.tools.set_credentials_file(username='dgrimshandl', api_key='••••••••••')

In [3]:
# path of the rydanalysis library
sys.path.insert(0, '/home/david/Institut/data-analysis/rydanalysis/')
from rydanalysis import *

In [4]:
old_data_root ='/home/david/Institut/2019_IEI'
data_root ='/home/david/Institut/2019_IEI_new'
analysis_root='/home/david/Institut/analysis'
seq_path = "October/22/OmegaPScanFinalFinalFinal/"

In [5]:
seq = ExpSequence(join(data_root,seq_path))

In [6]:
%%time
ds = seq.raw_data.get_images()

CPU times: user 31.7 s, sys: 3.46 s, total: 35.1 s
Wall time: 47.4 s


In [7]:
ref = ds['image_03']
atoms = ds['image_01']
dark = ds['image_05']

xslice=slice(10,90)
yslice=slice(10,380)

dark = dark[:,xslice,yslice]
ref = ref[:,xslice,yslice]

In [8]:
std_shot = ref.std(dim=['x','y'])
mean_shot = ref.mean(dim=['x','y'])

In [9]:
std = ref.groupby('pABSx').std(dim='shot')

In [10]:
std_seq = ref.std(dim='shot')
mean_seq = ref.mean(dim='shot')

In [11]:
std_p = ref.groupby('pABSx').std('shot')
mean_p = ref.groupby('pABSx').mean('shot')

In [12]:
intens = ref.mean(('x','y'))
intens_mean = intens.groupby('pABSx').mean('shot')
intens_std = intens.groupby('pABSx').std('shot')

In [13]:
bg = dark.mean()

In [14]:
fig = go.FigureWidget()

fig.add_scattergl(x=mean_shot,y=std_shot,mode='markers');
fig.add_scattergl(x=mean_shot,y=std_shot,mode='markers');
fig.add_scattergl(x=mean_shot,y=std_shot,mode='markers');


In [15]:
fhist = go.FigureWidget()
fhist.add_histogram(x=intens,nbinsx=400)
fhist.update_xaxes(title_text = 'accumulated intensity [counts/pixel]')
fhist.update_yaxes(title_text = '# shots')

FigureWidget({
    'data': [{'nbinsx': 400,
              'type': 'histogram',
              'uid': '9d067ef3-…

In [16]:
fig2 = go.FigureWidget()

fig2.add_scattergl(x=intens_mean-bg,y=intens_std,mode='markers')
fig2.update_xaxes(title_text="mean accumulated intensity [counts/pixel]");
fig2.update_yaxes(title_text="standard deviation [counts/pixel]")

FigureWidget({
    'data': [{'mode': 'markers',
              'type': 'scattergl',
              'uid': 'cdaf3…

In [17]:
fig.update_layout(xaxis_type="log", 
                  yaxis_type="log")
fig.update_xaxes(title_text="mean intensity [counts]");
fig.update_yaxes(title_text="standard deviation [counts]");

In [18]:
fig.data[0].y= std_p[:,52,185]
fig.data[0].x= mean_p[:,52,185]-bg
fig.data[0].name='single pixel'

In [19]:
fig.data[1].x= np.linspace(fig.data[2].x.min(),fig.data[2].x.max(),10)
fig.data[1].y= np.sqrt(np.linspace(max(fig.data[2].x.min(),0),fig.data[2].x.max(),10))
fig.data[1].mode='lines'
fig.data[1].name='$\sqrt{}$'

In [20]:
#fig.add_scatter(x=std_p[:,52,185],y=std_p[:,52,185])
fig.data[2].x=mean_p.mean(('x','y'))-bg
fig.data[2].y=std_p.mean(('x','y'))
fig.data[2].error_y=dict(array=std_p.std(('x','y')).values)
fig.data[2].mode='lines+markers'
fig.data[2].name='average over all pixels'
fig

FigureWidget({
    'data': [{'mode': 'markers',
              'name': 'single pixel',
              'type': 's…